In [1]:
import os
import openai

from langsmith import Client
from qdrant_client import QdrantClient

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

/Users/datapsycho/PythonProjects/EtoEAIBoocamp/aieng-bootcamp-defaut/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Doenload an example reference data point from LangSmith

In [2]:
client = Client()

In [3]:
dataset = client.read_dataset(
    dataset_name="rag-evaluation-dataset"
)

In [4]:
dataset

Dataset(name='rag-evaluation-dataset', description='Dataset for evaluating RAG pipeline', data_type=<DataType.kv: 'kv'>, id=UUID('3c4555ce-f243-498f-944f-0a48fb9f208c'), created_at=datetime.datetime(2025, 11, 16, 14, 24, 7, 661153, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2025, 11, 16, 14, 24, 7, 661153, tzinfo=datetime.timezone.utc), example_count=34, session_count=0, last_session_start_time=None, inputs_schema=None, outputs_schema=None, transformations=None, metadata={'runtime': {'sdk': 'langsmith-py', 'library': 'langsmith', 'runtime': 'python', 'platform': 'macOS-15.6.1-arm64-arm-64bit', 'sdk_version': '0.4.41', 'runtime_version': '3.12.9', 'langchain_version': '1.0.5', 'py_implementation': 'CPython', 'langchain_core_version': '1.0.4'}})

In [5]:
list(client.list_examples(dataset_id=dataset.id, limit=10))[0].inputs

{'question': 'Do you have any specific drone or aerial photography products in your available stock?'}

In [6]:
list(client.list_examples(dataset_id=dataset.id, limit=10))[0].outputs

{'ground_truth': 'No, there are no drone or aerial photography products listed in your current stock.',
 'reference_context_ids': [],
 'reference_descriptions': []}

In [9]:
list(client.list_examples(dataset_id=dataset.id, limit=10))[5].inputs

{'question': 'Which products can help with adding multiple connection ports for device charging and data?'}

In [13]:
list(client.list_examples(dataset_id=dataset.id, limit=50))[20].outputs

{'ground_truth': 'Yes, you can use the 40 Pcs Silicone Cable Ties for various cords and cables, and the SunplusTrade Heavy Duty Zip Ties for organizing larger bundles or outdoor setups.',
 'reference_context_ids': ['B0B55TZLBR', 'B0BR8Q2PXN'],
 'reference_descriptions': ['40 Pcs Silicone Cable Ties Reusable Holder Strap Cord Ties Adjustable Cable Straps Multipurpose Charging Cable Organizer for Fastening Cable Charging Cords Wires (Black, White, Gray, Pink, ) Ample Quantity and Different Colors: the package contains 40 pieces of silicone cable ties in 4 different colors, including black, white, gray and pink, and each color has 10 pieces, ample quantity and various colors to meet your diverse needs Reliable and Long Lasting: the cable straps are made of quality silicone material, which is soft and flexible, reliable and firm to use, not easy to fray, break or fade, reusable and long lasting Proper Size: the silicone cable organizer measures approx. 11 cm/ 4.3 inches in length and appro

In [14]:
reference_input = list(client.list_examples(dataset_id=dataset.id, limit=50))[20].inputs
reference_output = list(client.list_examples(dataset_id=dataset.id, limit=50))[20].outputs

### RAG Pipeline

In [15]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [16]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )

    return response.data[0].embedding


def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "similarity_scores": similarity_scores,
    }


def process_context(context):

    formatted_context = ""

    for id, chunk in zip(context["retrieved_context_ids"], context["retrieved_context"]):
        formatted_context += f"- {id}: {chunk}\n"

    return formatted_context


def build_prompt(preprocessed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt

def generate_answer(prompt):

    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}],
        temperature=0.5,
    )

    return response.choices[0].message.content


def rag_pipeline(question, qdrant_client, top_k=5):

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    final_result = {
        "answer": answer,
        "question": question,
        "retrieved_context_ids": retrieved_context["retrieved_context_ids"],
        "retrieved_context": retrieved_context["retrieved_context"],
        "similarity_scores": retrieved_context["similarity_scores"]
    }

    return final_result

In [17]:
rag_pipeline("Can I get some charger?", qdrant_client, top_k=5)

{'answer': 'Yes, there are several charger cables available:\n\n1. iPhone Charger Cord Lightning Cables, Original 2022 Upgraded 3-pack, 3ft each, Apple MFi Certified, compatible with iPhone models including 13, 12, 11, XR, X, SE, 8, 7, 6 Plus, iPad, and AirPods. (3-pack)\n\n2. 5 in 1 USB C to Multi Charging Cable, 10ft long, Apple MFi Certified, with Lightning, Type C, and Micro USB connectors, can charge three devices simultaneously. (1 cable)\n\n3. MUXA 6 Pack Colorful Nylon Lightning Cables in various lengths (3ft, 6ft, 10ft), Apple MFi Certified, compatible with a wide range of iPhones and iPads. (6-pack)\n\n4. GREPHONE 2 Pack USB C to Lightning Cable, 6ft long, Apple MFi Certified, fast charging compatible with iPhone 13/12/11 series and iPads. (2-pack)\n\n5. Mixblu Charger Cable Replacement for Fitbit Inspire 3, 2 pack, 3.3ft long, designed specifically for Fitbit Inspire 3.\n\nPlease let me know which type or brand you prefer or if you need a specific length or compatibility.',


### RAGAS metrics

In [18]:
from ragas.dataset_schema import SingleTurnSample
from ragas.metrics import IDBasedContextPrecision, IDBasedContextRecall, Faithfulness, ResponseRelevancy

ragas_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
ragas_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

/var/folders/tn/l_62dr1n6hzgchkrr465lqz40000gn/T/ipykernel_33752/168323522.py:4: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  ragas_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
/var/folders/tn/l_62dr1n6hzgchkrr465lqz40000gn/T/ipykernel_33752/168323522.py:5: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  ragas_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))


In [19]:
reference_input

{'question': 'Are there any products available that can help tidy up cable management in a workspace?'}

In [20]:
reference_output

{'ground_truth': 'Yes, you can use the 40 Pcs Silicone Cable Ties for various cords and cables, and the SunplusTrade Heavy Duty Zip Ties for organizing larger bundles or outdoor setups.',
 'reference_context_ids': ['B0B55TZLBR', 'B0BR8Q2PXN'],
 'reference_descriptions': ['40 Pcs Silicone Cable Ties Reusable Holder Strap Cord Ties Adjustable Cable Straps Multipurpose Charging Cable Organizer for Fastening Cable Charging Cords Wires (Black, White, Gray, Pink, ) Ample Quantity and Different Colors: the package contains 40 pieces of silicone cable ties in 4 different colors, including black, white, gray and pink, and each color has 10 pieces, ample quantity and various colors to meet your diverse needs Reliable and Long Lasting: the cable straps are made of quality silicone material, which is soft and flexible, reliable and firm to use, not easy to fray, break or fade, reusable and long lasting Proper Size: the silicone cable organizer measures approx. 11 cm/ 4.3 inches in length and appro

In [21]:
result = rag_pipeline(reference_input["question"], qdrant_client)

In [22]:
result

{'answer': 'Yes, there are products available that can help tidy up cable management in a workspace:\n\n1. The 40 Pcs Silicone Cable Ties Reusable Holder Strap Cord Ties (B0B55TZLBR) come in multiple colors and are adjustable, reusable, and versatile for organizing various cables and wires.\n\n2. The SunplusTrade Heavy Duty Plastic Wire Zip Ties (B0BR8Q2PXN) are strong, self-locking nylon zip ties that can hold up to 60lbs, suitable for organizing wires and cables both indoors and outdoors.\n\nBoth options are designed to help keep cables neat and organized in your workspace.',
 'question': 'Are there any products available that can help tidy up cable management in a workspace?',
 'retrieved_context_ids': ['B0B55TZLBR',
  'B0BR8Q2PXN',
  'B0BP9Z159S',
  'B09XCKYXR8',
  'B0BFPZGYLD'],
 'retrieved_context': ['40 Pcs Silicone Cable Ties Reusable Holder Strap Cord Ties Adjustable Cable Straps Multipurpose Charging Cable Organizer for Fastening Cable Charging Cords Wires (Black, White, Gray

In [23]:
async def ragas_faithfulness(run, example):

    sample = SingleTurnSample(
            user_input=run["question"],
            response=run["answer"],
            retrieved_contexts=run["retrieved_context"]
        )
    scorer = Faithfulness(llm=ragas_llm)

    return await scorer.single_turn_ascore(sample)

In [24]:
await ragas_faithfulness(result, "")

0.8

In [25]:
async def ragas_responce_relevancy(run, example):

    sample = SingleTurnSample(
            user_input=run["question"],
            response=run["answer"],
            retrieved_contexts=run["retrieved_context"]
        )
    scorer = ResponseRelevancy(llm=ragas_llm, embeddings=ragas_embeddings)

    return await scorer.single_turn_ascore(sample)

In [26]:
await ragas_responce_relevancy(result, "")

np.float64(0.9625139193448019)

In [27]:
async def ragas_context_precision_id_based(run, example):

    sample = SingleTurnSample(
            retrieved_context_ids=run["retrieved_context_ids"],
            reference_context_ids=example["reference_context_ids"]
        )
    scorer = IDBasedContextPrecision()

    return await scorer.single_turn_ascore(sample)

In [28]:
await ragas_context_precision_id_based(result, reference_output)

0.4

In [29]:
async def ragas_context_recall_id_based(run, example):

    sample = SingleTurnSample(
            retrieved_context_ids=run["retrieved_context_ids"],
            reference_context_ids=example["reference_context_ids"]
        )
    scorer = IDBasedContextRecall()

    return await scorer.single_turn_ascore(sample)

In [30]:
await ragas_context_recall_id_based(result, reference_output)

1.0